# Add Classification Model Predictions

In this recipe, we will be covering how you can add classification model predictions to your dataset. First we will go through how to add predictions using the [FiftyOne Model Zoo](https://docs.voxel51.com/user_guide/model_zoo/index.html) and `apply_model`. In the second part, we will demonstrate how to add your classification predictions from your own custom model. Feel free to skip ahead if you are interested in only custom models!

## Setup

If you haven't already, install FiftyOne:

In [ ]:
!pip install fiftyone

Let's kick things off by loading in the zoo dataset [imagenet-sample](https://docs.voxel51.com/user_guide/dataset_zoo/datasets.html#dataset-zoo-imagenet-sample). The dataset contains 1,000 images, one randomly chosen from each class of the validation split of the ImageNet 2012 dataset. We will use it our example dataset for the tutorial.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset("imagenet-sample")

session = fo.launch_app(dataset)

## Adding Predictions With Model Zoo

With FiftyOne, you have tons of pretrained models at your disposal to use via the [FiftyOne Model Zoo](https://docs.voxel51.com/user_guide/model_zoo/index.html) or using one of our [integrations](https://docs.voxel51.com/integrations/index.html) such as [HuggingFace](https://docs.voxel51.com/integrations/huggingface.html)! To get started using them, first load the model in and pass it into the `apply_model` function. We will use [mobilenet-v2](https://docs.voxel51.com/user_guide/model_zoo/models.html#mobilenet-v2-imagenet-torch) from the model zoo first!

In [ ]:
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
dataset.apply_model(model, label_field="zoo_predictions")

Next, lets try a transformer for image classification from HuggingFace. Check out [here](https://docs.voxel51.com/integrations/huggingface.html#image-classification) for more options for HuggingFace transformers for image classification! Afterwards, we can visualize our results and see our predictions in the app!

In [ ]:
from transformers import BeitForImageClassification

model = BeitForImageClassification.from_pretrained(
    "microsoft/beit-base-patch16-224"
)
dataset.apply_model(model, label_field="hf_predictions")

session.show()

![Add_predictions](../assets/add_predictions.png)

## Adding Predictions from a Custom Model

Often you may bring your own model to add classification predictions! Let's walkthrough how using `torchvision` resnet50 we can add classification labels!

In [4]:
# If you dont have torchvision installed
!pip3 install torch torchvision

We start by initializing our model from torchvision:

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()



Before we jump into adding predictions to the _whole_ dataset, we will start with a single sample first. The pattern for adding custom labels looks like this:

1. Load the sample image
2. Perform any necessary preprocessing
3. Inference on the image
4. Grab the prediction and confidence of the model_output
5. Add the values as a label to your sample

In [7]:
# Step 1: Load the image from the sample.filepath
sample = dataset.first()
img = read_image(sample.filepath)


# Step 2: Initialize the inference transforms
preprocess = weights.transforms()


# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

tench: 54.5%


We can add a new field easily by just using `sample["new_field"] = value`. We can add a [classification](https://docs.voxel51.com/user_guide/using_datasets.html#classification) label like below:

In [8]:
# Step 5: Add the label and confidence score to the sample
sample["custom_predictions"] = fo.Classification(label=category_name, confidence=score)
sample.save() # Always remember to save after!

print(sample.custom_predictions)

<Classification: {
    'id': '6642742f831fa59b38e3d03c',
    'tags': [],
    'label': 'tench',
    'confidence': 0.5446103811264038,
    'logits': None,
}>


Now that we know how to do a single sample, we can create a function that takes in our model and our dataset to add predictions to all samples! We copy the same pattern but iterate through our dataset this time to grab each sample:

In [10]:
def add_predictions(dataset, model):
    
    weights = ResNet50_Weights.DEFAULT
    preprocess = weights.transforms()
    
    for sample in dataset:
        
        img = read_image(sample.filepath)

        batch = preprocess(img).unsqueeze(0)

        prediction = model(batch).squeeze(0).softmax(0)
        class_id = prediction.argmax().item()
        score = prediction[class_id].item()
        category_name = weights.meta["categories"][class_id]

        sample["custom_predictions"] = fo.Classification(label=category_name, confidence=score)
        sample.save()

Now we can call our new function and visualize our results!

In [11]:
add_predictions(dataset,model)
session.show()